In [4]:
#este programa se puede utilizar para crear la base de datos

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

#getting video
cap_rbg = cv2.VideoCapture(0)
cap_depth = cv2.VideoCapture(1)

#tamaño de las imagenes capturadas
im_size = 28

#contamos numero de archivos en la carpeta
dir_name='dataset_28/2_fingers'
numfiles = len([f for f in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, f))])

#inicializamos cuadrado del roi
x,y,w,h = 0,0,100,100

#used for saving video
#fourcc = cv2.VideoWriter_fourcc(*'X264')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while True:
    
    #saving video in a variable
    ret, frame_rgb = cap_rbg.read()
    ret, frame_depth = cap_depth.read()
    
    #aplicamos efecto espejo
    frame_depth = cv2.flip(frame_depth,1)
    
    #selecting the background color for applying the mask
    lower_green = np.array([0, 154, 0])
    upper_green = np.array([0, 154, 0])
    mask = cv2.inRange(frame_depth, lower_green, upper_green)
    
    #noise filter
    median = cv2.medianBlur(mask ,15)
    
    #Invierto coloeres de la imagen para que funcione bien la captura de contorno
    median = 255 - median;
    
    #reducimos el tamaño de la imagen
    resized_image = cv2.resize(median, (200, 150))
    
    #dibujamos los rectangulos para la roi (fija) en las imagenes pequeña y grande
    #cv2.rectangle(median,(0,0),(200,200),(0,255,0),1)
    #cv2.rectangle(median,(440,0),(640,200),(0,255,0),1)
    #cv2.rectangle(resized_image,(0,0),(50,50),(0,255,0),1)
    #cv2.rectangle(resized_image,(110,0),(160,50),(0,255,0),1)
    
    #recortamos la roi (fija)
    #roi_left = resized_image[0:50,0:50]
    #roi_right = resized_image[0:50,110:160]
    
    #Buscamos el contorno en la imagen filtrada
    median, contours, hierarchy = cv2.findContours(median,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) != 0:
        # Dibujamos los contornos
        # cv2.drawContours(median, contours, -1, (122,122,0), 3)

        #Buscamos el area maxima
        c = max(contours, key = cv2.contourArea)
        
        #Declaramos las medidas del rectangulo de contorno maximo
        x,y,w,h = cv2.boundingRect(c)
        
        #Dibujamos el contorno
        cv2.rectangle(median,(x,y),(x+w,y+h),(255,255,255),2)
        
    roi_max = median[y:y+h,x:x+w]
    
    resized_roi = cv2.resize(roi_max, (im_size, im_size))
           
    #show videos   
    #cv2.imshow('frame_rgb', frame_rgb)
    cv2.imshow('frame_depth', frame_depth)
    #cv2.imshow('mask', mask)
    
    cv2.imshow('median', median)
    #cv2.imshow('resized_image', resized_image)
    #cv2.imshow('roi_left', roi_left)
    #cv2.imshow('roi_right', roi_right)
    cv2.imshow('resized_roi', resized_roi)
    cv2.imshow('roi_max', roi_max)
    
    #guardamos los frames de la mano izda en .jpg al pulsar el numero
    if cv2.waitKey(1) & 0xFF == ord('c'):
        name = "hand%d.png"%(numfiles + 1)
        cv2.imwrite(os.path.join(dir_name, name), resized_roi)
        numfiles += 1

    #salimos de la ejecucion con q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap_rbg.release()
cap_depth.release()
cv2.destroyAllWindows()
